In [2]:
#!ls /usr/bin/chromium-browser #path of chromium if needed to see packages

In [3]:
# Install necessary libraries, ****check for selenium and webdriver versions
!pip install selenium==4.9.1 webdriver-manager==3.8.6  beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 24.5 MB/s eta 0:00:00


In [4]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
0 upgraded, 9 newly installed, 0 to remove and 29 not upgraded.
Need to get 30.3 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3build1 [159 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-up

In [5]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,375 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,753 kB]
Get:13 http://archive.ubuntu.c

In [6]:
# Import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import time

In [7]:
def scrape_dynamic_data(url):
    """
    Scrapes dynamic data from a URL using Selenium and Beautiful Soup.

    Paremeters:
        url (str): The URL to scrape.

    Returns:
        tuple: A tuple containing the BeautifulSoup object and the HTTP status code,or (None, status_code) if the connection fails.
    """

    try:
        # Check HTTP status code
        response = requests.get(url)
        status_code = response.status_code

        if status_code != 200:
            print(f"HTTP request failed with status code: {status_code}")
            return None, status_code

        # Set up Selenium WebDriver
        options = webdriver.ChromeOptions()
        options.add_argument("--verbose")
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument("--window-size=1920, 1080")
        options.add_argument('--disable-dev-shm-usage')
        options.add.argument('--disable-extensions')
        options.add_argument('--remote-debugging-port=9222') #Crucial for Colab

        driver = webdriver.Chrome(service=Service('/usr/bin/chromedriver'), options=options) #In case it gives you error of path
        #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        time.sleep(5) #Add delay.
        driver.get(url)

        # Allow time for JavaScript to load (adjust as needed)
        driver.implicitly_wait(10)

        # Get the page source after JavaScript execution
        html_content = driver.page_source

        driver.quit()

        # Parse with Beautiful Soup
        soup = BeautifulSoup(html_content, 'html.parser')

        return soup, status_code

    except requests.exceptions.RequestException as e:
         print(f"Error during HTTP request: {e}")
         return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None


url = 'https://www.google.com/maps/place/Hilton+Head+Island+Airport/@32.224848,-80.697165,13z/data=!4m8!3m7!1s0x88fc70e9ba5935a7:0x376dbc2225a42676!8m2!3d32.2248476!4d-80.6971648!9m1!1b1!16zL20vMDdkdzdn?hl=es-419&entry=ttu&g_ep=EgoyMDI1MDMxMi4wIKXMDSoASAFQAw%3D%3D'  # Replace with your URL

#################################################################################
#Scraping phase

soup, status_code = scrape_dynamic_data(url) #Because the function retrieves a tuple: soup and status_code
response = requests.get(url)
print('Webpage response:',response.status_code)

#Data extraction
if soup:
    # Extract opinions
    opinions = []
    for opinion in soup.find_all('span', class_='wiI7pd'):
        opinions.append(opinion.text.strip())

    # Extract ratings
    ratings = []
    for rating in soup.find_all('span', class_='kvMYJc'):
        ratings.append(rating.get('aria-label'))
else:
    print(f"Failed to retrieve data. Status code of Browser: {status_code}")



An error occurred: 'Options' object has no attribute 'add'
Webpage response: 200
Failed to retrieve data. Status code of Browser: None


In [8]:
!chmod +x /usr/bin/chromedriver #driver has execute permissions if no error code is retrieved

In [9]:
#In case you are having troubles with chroumium
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/

--2025-03-15 14:01:40--  https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.210.207, 173.194.212.207, 173.194.215.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.210.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7407250 (7.1M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   7.06M  --.-KB/s    in 0.06s   

2025-03-15 14:01:40 (121 MB/s) - ‘chromedriver_linux64.zip’ saved [7407250/7407250]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


In [10]:
!apt-get update
!apt-get install -y xvfb libgtk-3-0 libnotify-dev libgconf-2-4 libnss3 libxss1 libasound2

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [11]:
!chromium-browser --version
!chromedriver --version


Command '/usr/bin/chromium-browser' requires the chromium snap to be installed.
Please install it with:

snap install chromium

ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [12]:
!sudo snap install chromium

error: cannot communicate with server: Post "http://localhost/v2/snaps/chromium": dial unix /run/snapd.socket: connect: no such file or directory
